In [1]:
import nltk #toolkit for NLP 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import nltk; nltk.download('popular')
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents1.json').read()
intents = json.loads(data_file)
print(intents)
#here the json.intents file is broken into words classes and documents


for intent in intents['intents']: #Intents catagorize the users intentions based off word patterns 
    for pattern in intent['patterns']: # pattern are word combinations that will incite a specific response 

        #tokenize each word
        w = nltk.word_tokenize(pattern) #here we vectorize the patterns with tokenize, that is we use a number identifier for each linguistic fragment
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
print(words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
# this is where the chatbot gets trained
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5)) #The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time, which helps prevent overfitting. Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all inputs is unchanged.
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax')) #softmax converts a vector of numbers into a vector of probabilities(0 to 1)

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)     #70/30 testing and training data
model.save('chatbot_model.h5', hist)

print("model created")

#___________________________________________________________

import nltk                                    #here we program the chat bot to understand human speech irregularity 
from nltk.stem import WordNetLemmatizer        #with the natural language tool kit (nltk)
lemmatizer = WordNetLemmatizer()
import pickle                                  #pickle converts a Python object into a byte stream to store it in a file/database
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))



def clean_up_sentence(sentence):   #helps to understand user even with spelling mistakes
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern (creates an array)
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):              #The predict class function helps to understand conversation subject 
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):  #using the "intents" file the chat bot can try to choose a correct response 
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):            #the last function triggers the window to print a response
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


#Creating GUI with tkinter (GRAPICAL USER INTERFACE)
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Chat Bot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command = send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |   

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

C:\Users\Ryan\AppData\Local\Temp/ipykernel_30448/3829260693.py:78: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
C:\Users\Ryan\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 1s 2ms/step - loss: 2.1684 - accuracy: 0.1702
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.2075 - accuracy: 0.1489
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0351 - accuracy: 0.3617
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9519 - accuracy: 0.3404
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.8740 - accuracy: 0.3830
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6174 - accuracy: 0.5745
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6008 - accuracy: 0.5319
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 1.5184 - accuracy: 0.5106
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3073 - accuracy: 0.6170
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.2000 - accuracy: 0.6596
Epoch 11/

Epoch 83/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0541 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0736 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 991us/step - loss: 0.0401 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0931 - accuracy: 0.9574
Epoch 87/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0341 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0438 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0603 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0397 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1227 - accuracy: 0.936

10/10 [==============================] - 0s 2ms/step - loss: 0.0606 - accuracy: 0.9787
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0611 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0292 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0189 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0162 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0139 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0146 - accuracy: 1.0000
Epoc